<a href="https://colab.research.google.com/github/csy99/dna-nn-theory/blob/master/viridae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install biopython
!pip install PyDrive

     |████████████████████████████████| 2.3MB 7.1MB/s 


In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
import numpy as np
import pandas as pd
from Bio import SeqIO
import tensorflow as tf
from tensorflow import keras

# Read Data

In [47]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [7]:
downloaded = drive.CreateFile({'id':"1cPtjYxlEkeKwm6FzkSkINXcUIv_TfJBQ"})
downloaded.GetContentFile('5UTR_aligned.fasta', mimetype=None)  

In [51]:
downloaded = drive.CreateFile({'id':"1SWt9LDKs9WYVM7i0jHs0dOtcUBuQHUZB"})
downloaded.GetContentFile('picornaviridae_5UTR_aligned.fas', mimetype=None)  

In [48]:
downloaded = drive.CreateFile({'id':"1UMJXRopZs6nQ_LiHdk0-w2QK6PlmXb3p"})
downloaded.GetContentFile('potyviridae_5UTR_aligned.fas', mimetype=None)  

In [ ]:
# # records_df = pd.DataFrame(columns=["id", "seq"])
# id_list = []
# seq_list = []
# with open("5UTR_aligned.fasta", "rU") as handle:
#   for record in SeqIO.parse(handle, "fasta"):
#     records_df = 
#     # print("id is", record.id, "seq is", record.seq)

In [39]:
def readFasta(file):
  with open(file, 'r') as f:
    records = np.array([record.seq._data.upper() for record in SeqIO.parse(f, 'fasta')])
  with open(file, 'r') as f:
    records_id = np.array([record.id for record in SeqIO.parse(f, 'fasta')])
  print('Number of sequences :', len(records))
  print('Length of sequences :', len(records[0]))
  records_df = pd.DataFrame({'id': records_id, 'seq': records})
  return records_df

In [70]:
records_df = readFasta('5UTR_aligned.fasta')
records_df.head()

Number of sequences : 283
Length of sequences : 4132


,id,seq
0,NC_036588.1,----------------------------------------------...
1,NC_014412.1,----------------------------------------------...
2,NC_014413.1,G---------------------------------------------...
3,NC_025890.1,----------------------------------------------...
4,NC_023162.1,----------------------------------------------...


In [52]:
picons_df = readFasta('picornaviridae_5UTR_aligned.fas')
picons_df.head()

Number of sequences : 129
Length of sequences : 2104


,id,seq
0,NC_036588.1_Passerivirus_sp._strain_waxbill/DB...,----------------------------------------------...
1,NC_028365.1_Hedgehog_hepatovirus_Igel8Erieur20...,----------------------------------------------...
2,NC_028364.1_Shrew_hepatovirus_KS121232Sorara20...,----------------------------------------------...
3,NC_028363.1_Rodent_hepatovirus_RMU101637Micarv...,----------------------------------------------...
4,NC_004421.1_Aichivirus_B_genomic_RNA_complete_...,----------------------------------------------...


In [49]:
potys_df = readFasta('potyviridae_5UTR_aligned.fas')
potys_df.head()

Number of sequences : 152
Length of sequences : 872


,id,seq
0,NC_036802.1_Saffron_latent_virus_isolate_Ir-Kh...,----------------------------------------------...
1,NC_018093.1_Sweet_potato_virus_G_isolate_Jesus...,----------------------------------------------...
2,NC_001841.1_Sweet_potato_feathery_mottle_virus...,----------------------------------------------...
3,NC_035461.1_Reed_chlorotic_stripe_virus_strain...,----------------------------------------------...
4,NC_035459.1_Sudan_watermelon_mosaic_virus_isol...,----------------------------------------------...


In [55]:
def truncateId(x, length=11):
  return x[:length]
picons_ids = list(picons_df.id.apply(truncateId))
potys_ids = list(potys_df.id.apply(truncateId))

In [71]:
# fill label with -1 and replace them later if found in two other reference files
records_df['label'] = pd.Series([-1 for x in range(len(records_df.index))], index=records_df.index)
records_df = records_df.set_index('id')
records_df.loc[picons_ids, 'label'] = 0
records_df.loc[potys_ids, 'label'] = 1
records_df.label.value_counts()

 1    152
 0    129
-1      2
Name: label, dtype: int64

In [72]:
# drop data which do not have correct label
invalid_idx = records_df[ records_df['label'] == -1 ].index
records_df.drop(invalid_idx, inplace=True)
records_df.label.value_counts()

1    152
0    129
Name: label, dtype: int64

In [84]:
# write to a csv file
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')
# Write the DataFrame to CSV file.
path = '/gdrive/My Drive/clean.csv'
with open(path, 'w') as f:
  records_df.to_csv(f)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [73]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(records_df.seq, records_df.label, test_size=0.2, random_state=100, stratify=records_df.label)
xtrain.shape, xtest.shape, ytrain.shape, ytest.shape

((224,), (57,), (224,), (57,))

In [77]:
ytrain.value_counts()

1    121
0    103
Name: label, dtype: int64

In [78]:
ytest.value_counts()

1    31
0    26
Name: label, dtype: int64

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts("ATCGN")
# number of distinct characters, should be 5 in this case
max_id = len(tokenizer.word_index) 
print(tokenizer.word_index)

{'a': 1, 't': 2, 'c': 3, 'g': 4, 'n': 5}


In [ ]:
batch_size = 8
kmer_len = 8
train_dataset = tf.data.Dataset.from_tensor_slices((xtrain, ytrain))

In [ ]:
# def seq2mat(x, tokenizer, kmer_len=8, shift=3): #, batch_size=32):
@tf.function
def seq2mat(x, y):
  # x = np.char.replace(x, '-', '')
  # x = np.char.replace(x, 'K', 'N')
  # x = np.char.replace(x, 'Y', 'N')
  # x = np.char.replace(x, 'R', 'N')
  # x = np.char.replace(x, 'W', 'N')
  # x = np.char.replace(x, 'M', 'N')
  # x = np.char.replace(x, 'S', 'N')

  x = tf.strings.regex_replace(x, "-", "")
  x = tf.strings.regex_replace(x, 'K', 'N')
  x = tf.strings.regex_replace(x, 'Y', 'N')
  x = tf.strings.regex_replace(x, 'R', 'N')
  x = tf.strings.regex_replace(x, 'W', 'N')
  x = tf.strings.regex_replace(x, 'M', 'N')
  x = tf.strings.regex_replace(x, 'S', 'N')

  # x = keras.layers.Flatten()(x)
  [x_encoded] = tokenizer.texts_to_sequences(x) - 1
  # x_dataset = tf.data.Dataset.from_tensor_slice(x_encoded)
  x_dataset = x_dataset.window(kmer_len, shift=3, drop_remainder=True)
  x_flat = x_dataset.flat_map(lambda window: window.batch(kmer_len))
  x_1hot = x_flat.map(lambda kmer: tf.one_hot(kmer, depth=max_id))
  
  # try to stack them into a matrix
  x_np_mat = []
  for item in x_1hot:
    line = np.array(item)
    x_np_mat.append(line.flatten())
  x_np_mat = np.array(x_np_mat)
  return x_np_mat

In [ ]:
train_data = train_dataset.shuffle(buffer_size=1000, seed=1)
train_data = train_data.map(seq2mat)
train_data = train_data.batch(batch_size).prefetch(1)

OperatorNotAllowedInGraphError: ignored

In [ ]:
# model = keras.models.Sequential([
#   keras.layers.GRU(128, return_sequences=True, 
#           input_shape=[None, max_id*kmer_len], 
#           dropout=0.2, recurrent_dropout=0.2),
#   keras.layers.GUR(128, return_sequences=True, 
#           dropout=0.2, recurrent_dropout=0.2), 
#   keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))                           
# ])


In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True, lower=True)
tokenizer.fit_on_texts("ATCGN")
x = []
seq = np.array(tokenizer.texts_to_sequences('ATCGATCN'))
a = keras.utils.to_categorical(seq[:,0]-1)
for i in a:
  x = x + list(i)
print(x)

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]


In [ ]:
seq[:, 0]

array([1, 2, 3, 4, 5])